# 概述
LangChain Expression Language 是langchain框架的表达式语言，用于描述和执行链式调用，通过声明式链式调用，可以轻松实现复杂的AI应用构建。它支持：流式处理、并行处理、回退处理、重试等高级功能。

## 安装

```bash
# 基础安装
pip install langchain_core
pip install langchain_openai

# 扩展安装 (需要根据需求安装)
pip install langchain_community
# openai 专用集成
pip install langchain_openai
# Anthropic 专用集成
pip install langchain_anthropic
# 谷歌gemini
pip install langchain_google_genai

# 所有依赖（开发环境推荐）
pip install langchain[all]
```
## 基础链式调用


In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('XIAOAI_API_KEY')
base_url = os.getenv('XIAOAI_BASE_URL')


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions."),
    ("user", "{input}"),
])
print(prompt)

llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name='claude-3-5-sonnet-20241022',
    temperature=0
)

chain = prompt | llm | StrOutputParser()

chain.invoke("给我讲讲RAG，并举个例子")


input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant that can answer questions.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


'好的,我来为您详细介绍RAG(Retrieval-Augmented Generation)。\n\nRAG是一种将检索(Retrieval)和生成(Generation)相结合的AI架构,主要用于增强大语言模型的知识储备和回答准确性。\n\nRAG的工作流程主要包含以下步骤:\n\n1. 知识库建设\n- 收集相关文档、文章等知识资料\n- 将文档分割成适当大小的文本块\n- 使用向量数据库存储文本块的向量表示\n\n2. 检索过程\n- 接收用户问题\n- 将问题转换为向量\n- 在向量数据库中检索相关文本\n- 选择最相关的若干段文本\n\n3. 生成过程\n- 将检索到的相关文本作为上下文\n- 结合问题生成最终答案\n\n举个具体例子:\n\n假设我们在开发一个医疗咨询机器人:\n\n1. 首先建立医学知识库:\n- 收集医学教材、论文、诊疗指南等资料\n- 将资料分割成小段并存入向量数据库\n\n2. 用户提问:"感冒发烧该怎么办?"\n\n3. 检索过程:\n系统会在知识库中检索出相关文段,如:\n- "感冒发烧是常见病症,建议多休息、多喝水..."\n- "当体温超过38.5度时,可以服用退烧药..."\n- "若持续高烧不退,需及时就医..."\n\n4. 生成回答:\n系统基于检索到的相关文本,生成类似这样的回答:\n"建议您:\n1) 多休息,保持充足睡眠\n2) 多喝温水,保持水分补充\n3) 如果体温超过38.5度,可以服用退烧药\n4) 若发烧持续不退或症状加重,请及时就医"\n\nRAG的优势:\n1. 提高回答准确性\n2. 可以获取最新知识\n3. 答案可溯源\n4. 降低模型幻觉\n\n使用场景:\n- 客服系统\n- 医疗咨询\n- 法律咨询\n- 教育辅导\n- 技术支持\n\n需要注意的是,RAG系统的效果很大程度上取决于:\n- 知识库的质量和覆盖范围\n- 检索算法的准确性\n- 向量化模型的效果\n- 生成模型的能力'

## 动态参数

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('XIAOAI_API_KEY')
base_url = os.getenv('XIAOAI_BASE_URL')


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions."),
    ("user", "{input}"),
])

llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name='claude-3-5-sonnet-20241022',
)
config = llm.with_config({"temperature": lambda x: 0.2 if x == "RAG" else 0})
chain = prompt | config | StrOutputParser()

# chain.invoke("给我讲讲RAG，并举个例子")
chain.invoke("我的是mac电脑，可以安装一个linux docker来训练大模型吗")


BadRequestError: Error code: 400 - {'error': {'message': 'messages: at least one message is required (request id: 20250303224501973471524KRIwhpBj) (request id: 20250303224501730281749ZrwHMD02) (request id: 20250303224501695699457mb3FTQYH)', 'type': 'invalid_request_error', 'param': '', 'code': None}}

## 分支处理


In [17]:
from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('XIAOAI_API_KEY')
base_url = os.getenv('XIAOAI_BASE_URL')



weather_prompt = PromptTemplate(input_variables=["location"], template="今天{location}的天气怎样?")
llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name='gpt-4-1106-preview'
)
weather_chain = weather_prompt | llm.with_config({"temperature": 0.5}) | StrOutputParser()

# Step 2: 创建新闻问题的模板和模型
news_prompt = PromptTemplate(input_variables=["topic"], template="我需要今天{topic}的新闻。")
news_chain = news_prompt | llm.with_config({"temperature": 0.1}) | StrOutputParser()

# Step 3: 创建聊天问题的模板和模型
chat_prompt = PromptTemplate(input_variables=["message"], template="让我们聊聊{message}。")
chat_chain = chat_prompt | llm.with_config({"temperature": 0.7}) | StrOutputParser()

# Step 4: 条件构建 Chain，根据输入选择不同的链
def choose_chain(user_input):
    if "天气" in user_input.lower():
        # 用户询问天气
        location = user_input.split()[-1]  # 假设用户输入最后一个词是地点
        return weather_chain.invoke({"location": location})
    elif "新闻" in user_input.lower():
        # 用户询问新闻
        topic = user_input.split()[-1]  # 假设用户输入最后一个词是新闻主题
        return news_chain.invoke({"topic": topic})
    else:
        # 闲聊
        return chat_chain.invoke({"message": user_input})

choose_chain("新闻俄乌战争")

'很抱歉，作为一个AI，我没有实时更新的能力，而且我的知识截止日期是2023年4月。因此，我无法提供今天的俄乌战争最新新闻。要获得最新信息，我建议查看可信赖的新闻网站、电视台或通过社交媒体平台关注官方新闻机构的实时更新。您也可以直接访问专业的新闻组织，如BBC、CNN、路透社、《纽约时报》等的网站，这些来源通常会及时报导国际新闻事件，包括俄乌战争的最新发展。请确保使用多个来源以获得全面和准确的信息。'

## 工具调用

In [ ]:
from langchain.tools import tool

@tool
def get_weather(city: str) -> str:
    """查询指定城市天气"""
    return f"{city}天气：25℃ 晴"

tools = [get_weather]
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions."),
    ("user", "{input}"),
])

load_dotenv()
api_key = os.getenv('XIAOAI_API_KEY')
base_url = os.getenv('XIAOAI_BASE_URL')

model = ChatOpenAI(base_url=base_url, api_key=api_key, model="gpt-4o-mini", temperature=0)
chain = (
    prompt_template 
    | model.bind_tools(tools)
    | ToolInvoke()
)

## 并行处理（同时处理多个任务）

In [ ]:
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('XIAOAI_API_KEY')
base_url = os.getenv('XIAOAI_BASE_URL')

model = ChatOpenAI(base_url=base_url, api_key=api_key, model="gpt-4o-mini", temperature=0)
model2 = ChatOpenAI(base_url=base_url, api_key=api_key, model="gpt-4o", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions."),
    ("user", "{topic}"),
])

parallel_chain = RunnableParallel(
    openai=model,
    claude=model2
)

result = parallel_chain.batch([{"topic": "狗"}, {"topic": "鱼"}])
